### Retrieve daily accummulated rainfall of select Wunderground PWS

This Jupyter notebook gathers the current daily accummulated rainfall of selected personal weather station from Wunderground. To be able to run this notebook properly, you should have an API key from Wunderground.

#### Importing necessary libraries

In [1]:
import os
import requests
import time
from datetime import datetime
import plotly.express as px
import pandas as pd
from pandas import json_normalize
from dotenv import find_dotenv, load_dotenv

### Get the API key and the list of PWS

In [2]:
# To be able to run this cell, you should have a .env file in your project folder containing the following:

# API_KEY='your_api_key_from_wunderground'
# PWS=STATION_ID_1, STATION_ID_2, STATION_ID_3, ..., STATION_ID_N

dotenv_path = find_dotenv()
load_dotenv(dotenv_path)

API_KEY = os.getenv("API_KEY")
PWS = os.getenv("PWS")

#### [OPTIONAL] This would allow showing all columns of a DataFrame if there are too many columns

In [3]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

#### Get the data from Wunderground via the API key

The data of each weather station would be appended in one DataFrame.

In [4]:
current_datetime = datetime.now().strftime("%Y%m%d_%H%M%S")
pws_list = PWS.split(", ")

for station in pws_list:
    url = "https://api.weather.com/v2/pws/observations/current?stationId=" + station + "&format=json&units=m&apiKey=" + API_KEY
    response = requests.get(url)

    try:
        data = response.json()
        df = json_normalize(data, record_path=['observations'])

        if station == pws_list[0]:
            dataset_df = df
        else:
            dataset_df = pd.concat([dataset_df, df], axis=0, ignore_index=True)

    except Exception as e:
        print(f"An error has occured: {e}")
    
    time.sleep(2)

In [5]:
# The "pointSize" column was added as a variable that will be used to dictate the size of markers in the map
dataset_df["pointSize"] = 5
dataset_df

,stationID,obsTimeUtc,obsTimeLocal,neighborhood,softwareType,country,solarRadiation,lon,realtimeFrequency,epoch,lat,uv,winddir,humidity,qcStatus,metric.temp,metric.heatIndex,metric.dewpt,metric.windChill,metric.windSpeed,metric.windGust,metric.pressure,metric.precipRate,metric.precipTotal,metric.elev,pointSize
0,IPARAAQU3,2023-08-14T14:30:02Z,2023-08-14 22:30:02,"Don Bosco Better Living, Paranaque City",weatherlink.com 1.10,PH,0.0,121.016617,None,1692023402,14.486938,0.0,213,88,1,29,36,27,29,10,10,1012.19,0.0,19.81,17,5
1,IPARAA10,2023-08-14T14:31:41Z,2023-08-14 22:31:41,Sun Valley,None,PH,0.0,121.023982,None,1692023501,14.495793,0.0,197,89,1,29,36,27,29,0,0,1012.87,0.0,27.43,4,5
2,IMETROMA22,2023-08-14T14:31:48Z,2023-08-14 22:31:48,Alabang,meteobridge,PH,0.0,121.039688,None,1692023508,14.440104,0.0,315,93,1,28,34,27,28,0,0,1011.18,0.0,1.78,20,5
3,IMUNTI6,2023-08-14T14:31:52Z,2023-08-14 22:31:52,Tunasan,None,PH,0.0,121.042118,None,1692023512,14.382696,0.0,221,91,1,29,36,27,29,2,2,1011.18,0.0,0.25,25,5
4,IMAKAT1,2023-08-14T14:31:52Z,2023-08-14 22:31:52,Poblacion,Cumulus v3.25.2,PH,0.0,121.027191,None,1692023512,14.570596,0.0,0,92,1,29,37,27,29,0,0,1011.24,0.0,0.51,2,5
5,IRIZBULA2,2023-08-14T14:31:54Z,2023-08-14 22:31:54,Village East Executive Homes Cainta,EasyWeatherV1.6.4,PH,0.0,121.122169,None,1692023514,14.598827,0.0,205,93,1,26,27,24,26,0,0,1014.12,0.0,48.79,13,5
6,IBULACAN2,2023-08-14T14:31:56Z,2023-08-14 22:31:56,Pandi Bulacan,HP1000 V1.3.2,PH,0.0,120.954842,None,1692023516,14.855423,0.0,68,99,-1,25,26,25,25,1,3,1007.79,0.0,11.94,31,5


##### Creating an HTML file showcasing the total precipitation map

Note: Aside from the total precipitation, you could also check other meteorological parameters. Check the comments in the cell below to see what inputs you should change to check other parameters.

In [6]:
# This figure is currently centered in Metro Manila, Philippines. Change the "center" depending on the locations of the weather stations
# you are checking.

fig = px.scatter_mapbox(data_frame=dataset_df,
                        lon=dataset_df["lon"],
                        lat=dataset_df["lat"],
                        size=dataset_df["pointSize"],
                        zoom=9,
                        width=1200,
                        height=800,
                        center={"lat": 14.5123, "lon": 121.0165},
                        text=dataset_df["stationID"],
                        color=dataset_df["metric.precipTotal"], # Modify this if you want other than total precipitation
                        color_continuous_scale="bluered",
                        labels={"metric.precipTotal": "Total Precipitation"},   # Modify this as well to change the labels
                        title="Total Precipitation Map")    # Modify as well

fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(dragmode=False)

# Modify the file name as well below to follow what parameter you would like to check
fig.write_html("selected_pws_total_precipitation_map_" + current_datetime + ".html")

# NOTE: This currently has an issue when text is defined in scatter_mapbox() above. To be addressed.
# fig.write_image("selected_pws_total_precipitation_map_" + current_datetime + ".png")